# Use OpenAPI with Managed Online Endpoints

This example demonstrates how to work with with OpenAPI and Managed Online Endpoints using both automatically-generated and custom Swagger files. 

The AzureML Inference Server automatically generates swagger files for scoring scripts that use [Inference Schema](https://github.com/Azure/InferenceSchema). In this example, a simple Inference Schema-decorated [scoring script](openapi/decorated/code/score.py) is used. For more complex examples, refer to the [Inference Schema example](../inference-schema).  

Managed Online Endpoints can alsoreturn user-defined swagger files.

## Prerequisites
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb)
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)


## 1. Connect to Azure Machine Learning

### 1.1 Import Required Libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
    BuildContext,
    ProbeSettings,
)
from azure.identity import DefaultAzureCredential
import requests, random

### 1.1 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.2 Set variables

In [ ]:
endpoint_name = f"endpt-moe-{random.randint(0,10000)}"

### 1.3 Create an MLClient instance

In [ ]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Create an endpoint

### 2.1 Define the endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)

### 2.2 Create the endpoint

In [ ]:
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 2.3 Get the endpoint's key

In [ ]:
key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

## 3. Create Deployment: Auto-Generated Swagger
When using Inference Schema, the AzureML Inference server will automatically create a Swagger file for you. In this deployment, the `code-decorated` folder contains only a `score.py` file without a user-supplied swagger file. The run function of this `score.py` file is decorated with Inference Schema decorators: 

```python 
@input_schema(
    param_name="data",
    param_type=NumpyParameterType(np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])),
)
@output_schema(output_type=StandardPythonParameterType({"output": [1.0, 1.0]}))
def run(data):
    logging.info("model 1: request received")
    result = model.predict(data)
    logging.info("Request processed")
    return {"output": result.tolist()}
``` 

### 3.1 Define the deployment


In [ ]:
deployment = ManagedOnlineDeployment(
    name="openapi",
    endpoint_name=endpoint_name,
    model=Model(path="../model-1/model"),
    code_configuration=CodeConfiguration(
        code="openapi/code-decorated", scoring_script="score.py"
    ),
    environment=Environment(
        image="mcr.microsoft.com/azureml/minimal-py312-inference",
        conda_file="openapi/env.yml",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

### 3.2 Create the deployment

In [ ]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

### 3.3 Set traffic to 100% 

In [ ]:
endpoint.traffic = {"openapi": 100}
endpoint = ml_client.begin_create_or_update(endpoint).result()

### 3.4 Test endpoint

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name, request_file="../model-1/sample-request.json"
)

### 3.5 Get Swagger (Default Version)

Swagger files are made available by default at the API endpoint `/swagger.json`. The specific route for an endpoint can be retrieved from the `openapi_uri` attribute of the endpoint. 

In [ ]:
res = requests.get(url=endpoint.openapi_uri, headers={"Authorization": f"Bearer {key}"})
print(res.content)

### 3.6 Get Swagger (Specify Version)
Specific versions can be retrieved by adding a `version` parameter to the request. 

In [ ]:
res = requests.get(
    url=endpoint.openapi_uri,
    params={"version": 3},
    headers={"Authorization": f"Bearer {key}"},
)
print(res.content)

## 4. Create Deployment: Custom Swagger

Custom swagger files can be integrated by including them at the root of the `code` directory. The custom file should be named `swagger<version>.json` i.e. `swagger2.json`. In this deployment, a user-supplied Swagger file named `swagger2.json` is located at the root of the `code-custom` directory alongside the `score.py` file:

```json
{
    "swagger": "2.0",
    "info": {
        "title": "ML service",
        "description": "A custom API description",
        "version": "1.0"
    },
    "schemes": [
        "https"
    ],
...
```

### 4.1 Update the deployment

In [ ]:
deployment.code_configuration = CodeConfiguration(
    code="openapi/code-custom", scoring_script="score.py"
)
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

### 4.2 Test endpoint

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name, request_file="../model-1/sample-request.json"
)

### 4.4 Get Custom Swagger
The custom swagger file we provided is supplied at `/swagger.json` even though the file is named `swagger2.json`. The requested version is controlled by the version parameter of the request, see section 3.7. 

In [ ]:
res = requests.get(url=endpoint.openapi_uri, headers={"Authorization": f"Bearer {key}"})
print(res.content)

## 5. Delete assets

### 5.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)